In [28]:
import pandas as pd
pd.options.display.max_columns=100
import kaggle
import pickle

import os
path=r"C:\Users\berid\python\yts mx project"

In [29]:
yts_df=pd.read_csv(os.path.join(path,'scraped_data','cleaned_yts_movies_df.csv'))
letterboxd_df=pd.read_csv(os.path.join(path,'scraped_data','cleaned_letterboxd_movies_df.csv'))

##### merging dataframes

In [30]:
letterboxd_df=letterboxd_df.dropna(subset='IMDB_LINK')
letterboxd_df['IMDB_LINK']=letterboxd_df['IMDB_LINK'].str.replace('http','https')

In [31]:
df=yts_df.merge(letterboxd_df,how='left',on='IMDB_LINK')

In [32]:
df['GENRES_x']=df['GENRES_x']+'|'
df['DIRECTOR_x']=df['DIRECTOR_x']+'|'
df['BIGRAMS_TAGS_x']=df['BIGRAMS_TAGS_x']+'|'
df['DIRECTOR_y']=df['DIRECTOR_y']+'|'
df['BIGRAMS_TAGS_y']=df['BIGRAMS_TAGS_y']+'|'

##### applying weights

In [33]:
all_tags_list=[]
for a,b,c,d,e,f,g,h in zip(df['GENRES_x'], df['DIRECTOR_x'], df['TAGS_x'], df['BIGRAMS_TAGS_x'], df['GENRES_y'], df['DIRECTOR_y'], df['TAGS_y'], df['BIGRAMS_TAGS_y']):
    all_tags=str(a)*12+str(b)*5+str(c)*3+str(d)*1+str(e)*1+str(f)*5+str(g)*10+str(h)*1

    all_tags_list.append(all_tags)

df['ALL_TAGS']=all_tags_list
df['ALL_TAGS']=df['ALL_TAGS'].apply(lambda x:x.replace('None','').replace('nan','').replace('Show All…','|').replace('||','|').replace('||','|'))

In [34]:
df=df[['IMDB_LINK','POSTER','TITLE_x','YEAR_x','GENRES_x','DIRECTOR_x','IMDB_RATING','IMDB_RATING_COUNT','ALL_TAGS']]
df.columns=df.columns.str.replace('_x','')

##### create similarity matrix

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

vectorizer = TfidfVectorizer(tokenizer=lambda x: x.split('|'), stop_words='english')
tfidf_matrix = vectorizer.fit_transform(df['ALL_TAGS'])

similarity_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

C:\Users\berid\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\feature_extraction\text.py:523: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [36]:
import numpy as np
import pickle

num_parts = 20
array_parts = np.array_split(similarity_matrix, num_parts)

for i, part in enumerate(array_parts):
    with open(os.path.join(path,'scraped_data','similarities_matrix',f'array_part_{i}.pkl'), "wb") as f:
        pickle.dump(part, f)